# Numerical test for response functions

To run this notebook, make sure to download the file **response.py**, where some functions for numerical integrations are defined.

In [28]:
import healpy as hp
import itertools
import jax
import jax.numpy as jnp
from jax import vmap, grad
from response import pairwise_monopole, pairwise_dipole, dir,pulsar_star_pair_monopole, pulsar_star_pair_dipole ,vvec 
from jax import config
# config.update("jax_enable_x64", True)
import numpy as np

# plot settings
import matplotlib
import matplotlib.pyplot as plt
np.set_printoptions(precision=4,suppress=True)
jnp.set_printoptions(precision=4,suppress=True)

jax.config.update("jax_enable_x64", True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


## PTA-Astrometry correlation

The first block contains elements for a numerical integration on the sphere for equation 2.22 in 2412.14010, while the second calculates $K_{i}^{(1)}$ with the analytical results reported in the same paper.

In [46]:
# ----------------- Numerical calculations ----------------------

# Defining vectors p over the sphere:
nside = 128
npix = hp.nside2npix(nside)
pix = np.arange(npix)
pvec_array = np.array(hp.pix2vec(nside,pix)).T



# Choosing different values for the vectors n and x:

nvec1 = np.array([0.96388439, -0.26566278, 0.01871302]).T # Random position in the Sky (close to the centre)
#nvec1 = vvec # vvec is defined in response.py
#nvec1 = np.array([1, 0, 0]).T

xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T # Random position in the Sky
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T


# Performing calculations with a function defined in response.py
dipole = pulsar_star_pair_dipole(nvec1,xvec1,pvec_array,nside)

print("Numerical dipole: Ki = ", dipole)



# ----------------- Analytical calculations ----------------------

vvec2 = vvec 
#vvec2 = np.array([0.6, 0.4, 0.69282]).T

nvec2 = np.array([0.96388439, -0.26566278, 0.01871302]).T # Random position in the Sky (close to the centre)
#nvec2 = vvec # When using this value, be aware of doing the calculations with dipole_analytical_Vparallel defined below
#nvec2 = np.array([0, 0, 1]).T

xvec2 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T # Random position in the Sky
#xvec2 = np.array([1/2, 0, (np.sqrt(3)/2)]).T


def dipole_analytical(nvec,xvec,vvec=vvec2):
    ndotx = jnp.sum(nvec*xvec)
    y = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)

    A1 = jnp.cross(nvec, xvec)
    A2 = jnp.cross(nvec, vvec)
    A1dotv = jnp.sum(A1*vvec)
    A1dotA2 = jnp.sum(A1*A2)

    def get_a1():
        num1 = jnp.pi*( (A1dotv**2)*(1 - 12*y) + A1dotA2*((A1dotA2) + 12*(A1dotA2)*(y**2) + 4*(ndotv)*y*(1 + y*(5 - 6*y)) )) 
        den1 = 6*A1dotv*(1 - y)*y
        num2 = 2*np.pi*( (A1dotA2**2) - (A1dotv**2) + 2*A1dotA2 *(ndotv)* (1 - y))*y*jnp.log(y)
        den2 = A1dotv* ((1-y)**2)
        return (num1/den1) + (num2/den2) 

    def get_a2():
        num3 = -2*jnp.pi*( A1dotA2 + 12*A1dotA2*(y**2)  + 4*ndotv*y*(1 + y*(5 - 6*y)) ) 
        den3 = 3*A1dotv
        num4 = -8*jnp.pi*( A1dotA2 + 2*(ndotv)*(1 - y) ) *((y**2) *jnp.log(y))
        den4 = A1dotv*(1 - y)
        return (num3/den3) + (num4/den4)       

    a1 = get_a1()
    a2 = get_a2()
    #print('a1, a2:', a1,a2)
    #print('A1, A2:', A1,A2)
    K_i = (a1*A1) + (a2*A2)
    return K_i


# ----------- For the CASE: v parallel to n: ------------

def dipole_analytical_Vparallel(nvec,xvec,vvec=vvec2):
    ndotx = jnp.sum(nvec*xvec)
    z = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)
    
    term1 = 2*np.pi*(nvec*(1 - 2*z) - xvec)/ (3* (1 - z))
    term2 = ((z - 1)*(6*z + 1) - 6*z* np.log(z))
    return term1*term2
# -------------------------------------------------------

dipole_gt = dipole_analytical(nvec2,xvec2)
print("Analytical dipole: Ki = ",dipole_gt)

# Uncomment the next lines for v parallel to n
#dipole_GTvParallel = dipole_analytical_Vparallel(nvec2,xvec2)
#print("Analytical dipole (v parallel to n) = ",dipole_GTvParallel)


Numerical dipole: Ki =  [ 0.3825  1.3555 -0.4599]
Analytical dipole: Ki =  [ 0.3825  1.3555 -0.4599]


## Astrometry - Astrometry correlation

The next cell contains calculations for $H_{ij}^{(1)}$ shown in 2412.14010

In [55]:
vvec1 = vvec
#vvec1 = np.array([0.6, 0.4, 0.69282]).T

nvec1 = np.array([0.96388439, -0.26566278, 0.01871302]).T
#nvec1 = np.array([0, 0, 1]).T
#nvec1 = np.array([1, 0, 0]).T
#nvec1 = -0.99999*vvec

xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T
#xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T

# -------------- Numerical integration -------------------
Hdipole = pairwise_dipole(nvec1,xvec1,pvec_array,nside)

print("Numerical dipole: H_ij = ", Hdipole)



# -------------- Analytical results -------------------
def Hdipole_analytical(nvec,xvec,vvec=vvec1):
    ndotx = jnp.sum(nvec*xvec)
    y = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)
    
    A = jnp.cross(nvec,xvec)
    B = jnp.cross(nvec, A)
    C = -jnp.cross(xvec, A)
    Adotv = jnp.sum(A*vvec)
    AdotA = jnp.sum(A*A)
    BdotB = jnp.sum(B*B)
    CdotC = jnp.sum(C*C)

    def get_a1():
        num = 8 *jnp.pi*(Adotv)*y* ( (3*(y**2)*jnp.log(y) / (y - 1)) - 4*y + 1)
        den = 3*(AdotA)*(CdotC)
        return num/den

    def get_a2():
        num = 8*jnp.pi*(Adotv)*y*( (3*(y**2)*jnp.log(y) / (y - 1)) - 4*y + 1)
        den = 3*(AdotA)*(BdotB)
        return num/den
        
    def get_a3():
        num = -8*jnp.pi*(y**2)*( (y - 1)*(2*y + 1) - 3*y*jnp.log(y)) * (2*ndotv*(y - 1) - jnp.sqrt( 4*( ndotv**2 - 1)*(y - 1)*y - Adotv**2))
        den = 3*(y - 1)*(AdotA**2)
        return num/den 

    def get_a4():
        num = (8*np.pi*(y**2)*( (y - 1)*(2*y + 1) - 3*y*jnp.log(y)) * 
          ( np.sqrt( ((Adotv**2)/(y - 1)) - 4*(ndotv**2 - 1)*y ) + 2*ndotv*np.sqrt(1 - y)))
        den = 3*np.sqrt(1 - y)*BdotB*CdotC
        return (num/den) 

    a1 = get_a1()
    a2 = get_a2()
    a3 = get_a3()
    a4 = get_a4()
    #print('a1, a2:', a1,a2)
    #print('AdotA:', AdotA)
    H_ij = a1*np.outer(A, C) + a2*np.outer(B, A) + a3*np.outer(A, A) + a4*np.outer(B, C)
    return H_ij


Hdipole_gt = Hdipole_analytical(nvec1,xvec1)

print("Analytical dipole: H_ij = ", Hdipole_gt)


Numerical dipole: H_ij =  [[ 0.147  -0.0231 -0.0848]
 [ 0.5358 -0.0387 -0.3094]
 [ 0.0377  0.6415 -0.0217]]
Analytical dipole: H_ij =  [[ 0.147  -0.0231 -0.0848]
 [ 0.5358 -0.0387 -0.3094]
 [ 0.0377  0.6415 -0.0217]]
